# Examen de laboratorio de APR 2023/24

## Preliminares

Estimación del error de regresión logística en una tarea de openml:

In [12]:
import warnings; warnings.filterwarnings("ignore"); import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

data_id = 3; X, y = fetch_openml(data_id=data_id, return_X_y=True, as_frame=False)
mask = ~np.isnan(X).any(axis=1); X = X[mask, :]; y = y[mask]
y = LabelEncoder().fit_transform(y); C = len(np.unique(y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
clf = LogisticRegression(random_state=23).fit(X_train, y_train)
N, D = X_train.shape; acc = accuracy_score(y_test, clf.predict(X_test))
print(f'Data id: {data_id}  N: {N}  D: {D}  C: {C}  Accuracy: {acc:.1%}')

Data id: 3  N: 2556  D: 36  C: 2  Accuracy: 94.5%


## Ejercicio

Realiza experimentos con el fin de hallar una red de error mínimo en la tarea anterior, usando la misma partición de los datos.

In [11]:
# crear modelo y entrenar
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Input
from keras.layers import Dense, Input
from keras.optimizers import Adam,SGD

In [13]:

model = Sequential()

dense = 36
batch = 32
lr=0.025

num_classes = 2 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model.add(Input(36))
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

sgd=SGD(learning_rate=lr,momentum=0.9)

model.compile(loss='categorical_crossentropy',
            optimizer=sgd,
            metrics=['accuracy'])

checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
batch_size=batch
epochs=25

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test), 
                    callbacks=[checkpoint])

# guardar modelo
model.save(f"model_.h5")

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model(f'best_model.h5')
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               18944     
                                                                 
 dense_15 (Dense)            (None, 2)                 1026      
                                                                 
Total params: 19970 (78.01 KB)
Trainable params: 19970 (78.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/25
67/80 [========================>.....] - ETA: 0s - loss: 0.4291 - accuracy: 0.7971
Epoch 1: val_accuracy improved from -inf to 0.93437, saving model to best_model.h5
80/80 [==============================] - 1s 6ms/step - loss: 0.3994 - accuracy: 0.8134 - val_loss: 0.2030 - val_accuracy: 0.9344
Epoch 2/25
66/80 [=======================>......] - ETA: 0s - loss: 0.1622 - accuracy: 0.9451
Epoch 2: val_a